In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment

In [2]:
MOT = pd.read_csv("../ADL-Rundle-6\det\Yolov5s\det.csv")
MOT.head()

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1695,386,136,342,0.760947,-1,-1,-1
1,1,-1,255,459,103,246,0.676300,-1,-1,-1
2,1,-1,1885,380,34,202,0.628092,-1,-1,-1
3,1,-1,2,306,81,614,0.575901,-1,-1,-1
4,1,-1,1284,463,74,195,0.575504,-1,-1,-1


In [3]:
# Code CC CV depuis internet https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] <= bb1['x2']
    assert bb1['y1'] <= bb1['y2']
    assert bb2['x1'] <= bb2['x2']
    assert bb2['y1'] <= bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [4]:
# Donne les information bien ranger depuis un DF pour la matrice IoU
def GetDictFromDF(liste,pos):
    liste = liste.iloc[pos].to_list()
    return {"x1":liste[2] ,"y1":liste[3] , "x2":liste[2]  + liste[4] ,"y2":liste[3] + liste[5]}

In [5]:
# Rend un matrice carré en rajoutant une ligne ou un colonne
def make_square(matrix):
    rows, cols = matrix.shape
    if rows == cols:
        return matrix
    max_dim = max(rows, cols)
    square_matrix = np.zeros((max_dim, max_dim), dtype=matrix.dtype)
    square_matrix[:rows, :cols] = matrix
    return square_matrix

##### pre-trained lightweight deep learning model 

In [6]:
import torch
from torchvision import models

# Charger un modèle léger pré-entraîné pour ReID
def load_reid_model():
    model = models.mobilenet_v2(pretrained=True)
    model.classifier = torch.nn.Identity()  
    model.eval()  # Mode évaluation
    return model

reid_model = load_reid_model()

c:\Users\lylia\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lylia\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
import cv2
import numpy as np

class PatchPreprocessor:
    def __init__(self, roi_width=64, roi_height=128, roi_means=(123.675, 116.28, 103.53), roi_stds=(58.395, 57.12, 57.375)):
        self.roi_width = roi_width
        self.roi_height = roi_height
        self.roi_means = roi_means
        self.roi_stds = roi_stds

    def preprocess_patch(self, im_crop):
        # Redimensionner le patch
        roi_input = cv2.resize(im_crop, (self.roi_width, self.roi_height))
        # Convertir de BGR à RGB
        roi_input = cv2.cvtColor(roi_input, cv2.COLOR_BGR2RGB)
        # Normaliser
        roi_input = (np.asarray(roi_input).astype(np.float32) - self.roi_means) / self.roi_stds
        # Réorganiser les dimensions (HWC -> CHW)
        roi_input = np.moveaxis(roi_input, -1, 0)
        return roi_input.astype('float32')

# Initialiser le préprocesseur
preprocessor = PatchPreprocessor()


In [8]:
def extract_reid_features(reid_model, preprocessed_patches):
    """
    reid_model : Modèle pré-entraîné pour ReID
    preprocessed_patches : Liste de patches prétraités
    """
    with torch.no_grad():  # Pas de calcul de gradients
        if len(preprocessed_patches.shape) == 3:
            preprocessed_patches = np.expand_dims(preprocessed_patches, axis=0)
        patches_tensor = torch.tensor(preprocessed_patches).to(torch.float32)  # Convertir en tenseur
        features = reid_model(patches_tensor)  # Obtenir les vecteurs de caractéristiques
    return features.numpy()



def GetFrame(framePos):
    
    frame_path = f"../ADL-Rundle-6/img1/000{str(framePos).zfill(3)}.jpg"
    frame = cv2.imread(frame_path)
    
    return frame

def GetCropInfos(CurrentFrame, i):
    frame = GetFrame(CurrentFrame)
    Left = int(MOT[MOT["frame"] == CurrentFrame].iloc[i][2])
    Top = int(MOT[MOT["frame"] == CurrentFrame].iloc[i][3])
    width = int(MOT[MOT["frame"] == CurrentFrame].iloc[i][4])
    height = int(MOT[MOT["frame"] == CurrentFrame].iloc[i][5])

    im_crop = frame[Top:Top+height, Left:Left+width]
    return extract_reid_features(reid_model, preprocessor.preprocess_patch(im_crop))

In [9]:
from scipy.spatial.distance import euclidean

def find_closest_indices(list1, list2):
    closest_indices = []
    for sublist1 in list1:
        sublist1_flat = sublist1.flatten()
        distances = [
            euclidean(sublist1_flat, sublist2.flatten()) for sublist2 in list2
        ]
        closest_index = distances.index(min(distances))
        closest_indices.append(closest_index)
    return closest_indices

In [10]:
# Initialisation des variables
CurrentFrame = 2  # Définir le cadre actuel
df = []  # Liste vide pour une future utilisation
debug = True  # Mode debug activé
nbFrames = len(MOT["frame"].unique())  # Nombre total de frames uniques dans les données MOT
posID = 0  # Position ID pour suivre les indices dans la liste des ID

# Initialisation des ID pour le premier frame
IDList = [1] * len(MOT[MOT["frame"] == CurrentFrame-1])  # Liste initiale des IDs pour les objets du frame précédent
for i in range(1, len(IDList)):  # Attribution d'IDs séquentiels
    IDList[i] += IDList[i-1]
    
IDMax = max(IDList)  # Maximum ID assigné jusqu'à présent
IDList = [IDList]  # Stockage des étapes successives des IDs
LastFrameCropInfos = []  # Infos de découpage (crop) pour le dernier frame traité

# Collecte des informations de découpage (crop) pour le frame précédent
for i in range(len(MOT[MOT["frame"] == CurrentFrame-1])):
    LastFrameCropInfos.append(GetCropInfos(CurrentFrame-1, i))

# Boucle principale pour traiter chaque frame jusqu'à nbFrames
while(CurrentFrame != nbFrames):
    CurrentTrackedList = MOT[MOT["frame"] == CurrentFrame-1]  # Objets du frame actuel
    NewTrackedList = MOT[MOT["frame"] == CurrentFrame]  # Objets du prochain frame
    MatriceIoU = []  # Matrice d'Intersection over Union (IoU)
    CurrentFrameDLinfos = []  # Informations sur les objets détectés dans le frame actuel

    # Construction de la matrice IoU
    for lig in range(len(NewTrackedList)):
        CurrentImageInfos = GetCropInfos(CurrentFrame, lig)  # Infos pour l'objet courant
        CurrentFrameDLinfos.append(CurrentImageInfos)  # Ajout des infos à la liste
        
        MatriceIoU.append([])  # Nouvelle ligne dans la matrice IoU
        value1forIoU = GetDictFromDF(NewTrackedList, lig)  # Infos objet courant
        for col in range(len(CurrentTrackedList)):
            MatriceIoU[lig].append([])  # Nouvelle colonne dans la matrice IoU
            value2forIoU = GetDictFromDF(CurrentTrackedList, col)  # Infos objet précédent
            MatriceIoU[lig][col] = get_iou(value1forIoU, value2forIoU)  # Calcul de l'IoU

    # Conversion et ajustement de la matrice IoU pour algorithme de correspondance
    MatriceIoU = np.array(MatriceIoU)
    Mshape = MatriceIoU.shape  # Dimensions initiales de la matrice IoU
    MatriceIoU = make_square(MatriceIoU)  # Assure que la matrice est carrée
    matcpy = MatriceIoU.copy()  # Copie pour traitement
    maximum = matcpy.max()
    matcpy = maximum - matcpy  # Inversion pour l'algorithme de correspondance
    poxlist = linear_sum_assignment(matcpy)[1]  # Meilleures correspondances via Hungarian Algorithm
    
    # Récupération des indices les plus proches avec Deep Learning
    indicesFromDL = find_closest_indices(CurrentFrameDLinfos, LastFrameCropInfos)

    # Gestion des cas où la matrice IoU est rectangulaire
    if Mshape[0] > Mshape[1]:
        pl = []
        for i in range(len(poxlist)):
            if MatriceIoU[i][poxlist[i]] != 0:
                pl.append(poxlist[i])
            else:
                pl.append(-1)  # Aucun match trouvé
        poxlist = pl

    # Mise à jour des IDs pour les objets détectés
    newIDList = [0] * len(poxlist)
    for i in range(len(poxlist)):
        if poxlist[i] < len(IDList[posID]) and poxlist[i] != -1:
            newIDList[i] = IDList[posID][poxlist[i]]
        else:
            newIDList[i] = IDMax  # Nouveau ID pour objet non associé
            IDMax += 1
            
    # Debugging des nouveaux IDs et associations
    print("---------")
    print(newIDList)
    print(poxlist)

    # Mise à jour pour la prochaine itération
    posID += 1
    IDList.append(newIDList)
    LastFrameCropInfos = CurrentFrameDLinfos
    CurrentFrame += 1

C:\Users\lylia\AppData\Local\Temp\ipykernel_27116\808088118.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Left = int(MOT[MOT["frame"] == CurrentFrame].iloc[i][2])
C:\Users\lylia\AppData\Local\Temp\ipykernel_27116\808088118.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Top = int(MOT[MOT["frame"] == CurrentFrame].iloc[i][3])
C:\Users\lylia\AppData\Local\Temp\ipykernel_27116\808088118.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `s

---------
[2, 1, 3, 4, 5, 6, 5, 7]
[1, 0, 2, 3, -1, -1, 4, -1]
---------
[2, 1, 3, 4, 6, 5, 5, 8, 7]
[0, 1, 2, 3, 5, 4, 6, -1, 7]
---------
[2, 4, 1, 3, 6, 8, 5, 5, 7]
[0 3 1 2 4 7 6 5 8]
---------
[2, 4, 6, 3, 1, 5, 7, 8, 5]
[0 1 4 3 2 6 8 5 7]
---------
[4, 2, 1, 3, 6, 5, 7]
[1 0 4 3 2 5 6]
---------
[4, 2, 1, 3, 6, 5, 7]
[0 1 2 3 4 5 6]
---------
[4, 2, 1, 3, 5, 6]
[0 1 2 3 5 4]
---------
[4, 1, 2, 3, 5, 6]
[0 2 1 3 4 5]
---------
[4, 1, 3, 2, 5, 6]
[0 1 3 2 4 5]
---------
[4, 1, 2, 3, 5, 9]
[0, 1, 3, 2, 4, -1]
---------
[4, 1, 2, 3, 5, 9]
[0 1 2 3 4 5]
---------
[4, 1, 3, 10, 5, 2]
[0, 1, 3, -1, 4, 2]
---------
[4, 1, 3, 10, 5, 2]
[0 1 2 3 4 5]
---------
[4, 1, 5, 3, 11, 10]
[0, 1, 4, 2, -1, 3]
---------
[4, 1, 5, 3, 11, 10]
[0 1 2 3 4 5]
---------
[4, 1, 3, 5, 11]
[0 1 3 2 4]
---------
[4, 1, 3, 5, 11]
[0 1 2 3 4]
---------
[4, 1, 3, 5, 11, 12]
[0, 1, 2, 3, 4, -1]
---------
[4, 3, 1, 5, 12, 11, 13, 14]
[0, 2, 1, 3, 5, 4, -1, -1]
---------
[4, 3, 1, 12, 5, 13, 11, 14]
[0 1 2 4 3 6 

In [11]:
import cv2
import os

frames_folder = "../ADL-Rundle-6/img1/"

# Lire toutes les frames du dossier
frames = sorted([os.path.join(frames_folder, f) for f in os.listdir(frames_folder) if f.endswith('.jpg')])

# Paramètres pour l'enregistrement de la vidéo
output_file = "output_video.avi"  # Nom du fichier de sortie
fps = 25  # Images par seconde
frame_size = None  # À définir après avoir lu la première frame

# Initialiser le VideoWriter (sera configuré après avoir lu une frame)
video_writer = None

framepos = 1
for frame_path in frames:
    # Charger l'image
    frame = cv2.imread(frame_path)
    height, width = frame.shape[:2]
    
    # Initialiser VideoWriter après avoir déterminé la taille des frames
    if video_writer is None:
        frame_size = (width, height)
        video_writer = cv2.VideoWriter(
            output_file,
            cv2.VideoWriter_fourcc(*'XVID'),  # Codec pour .avi
            fps,
            frame_size
        )
    
    rectangles = []
    for i in range(len(MOT[MOT["frame"] == framepos])):
        rectangles.append(GetDictFromDF(MOT[MOT["frame"] == framepos], i))
    
    assert len(rectangles) == len(MOT[MOT["frame"] == framepos])
    
    # Ajouter des rectangles
    posrec = 0
    for rect in rectangles:
        x1, y1, x2, y2 = int(rect["x1"]), int(rect["y1"]), int(rect["x2"]), int(rect["y2"])  # Regarder par rapport aux axes
        color = (0, 255, 0)
        thickness = 2
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 3
        font_thickness = 3
        text_color = (255, 0, 0)
        if framepos >= len(IDList) - 1:
            break
        print(framepos)
        print(posrec)
        print(len(rectangles))
        print(IDList[framepos - 1])
        print("---------")
        title = str(IDList[framepos - 1][posrec])
        cv2.putText(frame, title, (x1, y1), font, font_scale, text_color, font_thickness, cv2.LINE_AA)
        cv2.putText(frame, str(framepos), (30, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, str(len(MOT[MOT["frame"] == framepos])), (30, 70), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        posrec += 1
    
    framepos += 1
    
    # Écrire la frame dans le fichier vidéo
    video_writer.write(frame)
    
    # Afficher la frame
    cv2.imshow('Video Frame', frame)
    
    TimeToweight = 40
    # Attendre 30 ms pour simuler un FPS de ~30
    if cv2.waitKey(TimeToweight) & 0xFF == ord('q'):  # Quitter avec 'q'
        break

# Libérer les ressources
video_writer.release()
cv2.destroyAllWindows()


1
0
5
[1, 2, 3, 4, 5]
---------
1
1
5
[1, 2, 3, 4, 5]
---------
1
2
5
[1, 2, 3, 4, 5]
---------
1
3
5
[1, 2, 3, 4, 5]
---------
1
4
5
[1, 2, 3, 4, 5]
---------
2
0
8
[2, 1, 3, 4, 5, 6, 5, 7]
---------
2
1
8
[2, 1, 3, 4, 5, 6, 5, 7]
---------
2
2
8
[2, 1, 3, 4, 5, 6, 5, 7]
---------
2
3
8
[2, 1, 3, 4, 5, 6, 5, 7]
---------
2
4
8
[2, 1, 3, 4, 5, 6, 5, 7]
---------
2
5
8
[2, 1, 3, 4, 5, 6, 5, 7]
---------
2
6
8
[2, 1, 3, 4, 5, 6, 5, 7]
---------
2
7
8
[2, 1, 3, 4, 5, 6, 5, 7]
---------
3
0
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
3
1
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
3
2
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
3
3
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
3
4
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
3
5
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
3
6
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
3
7
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
3
8
9
[2, 1, 3, 4, 6, 5, 5, 8, 7]
---------
4
0
9
[2, 4, 1, 3, 6, 8, 5, 5, 7]
---------
4
1
9
[2, 4, 1, 3, 6, 8, 5, 5, 7]
---------
4
2
9
[2, 4, 1, 3, 6, 8, 5, 